In [1]:
import numpy as np

from manipulation import running_as_notebook

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    DiagramBuilder,
    ModelVisualizer,
    MultibodyPlant,
    BodyIndex,
    InverseKinematics,
    RotationMatrix,
    SceneGraph,
    Simulator,
    RollPitchYaw,
    InverseDynamicsController,
    BsplineTrajectory,
    KinematicTrajectoryOptimization,
    SpatialVelocityConstraint,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MinimumDistanceLowerBoundConstraint,
    Parser,
    PositionConstraint,
    Rgba,
    RigidTransform,
    Role,
    Solve,
    Sphere,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad, PublishPositionTrajectory
from manipulation.utils import ConfigureParser
from centers import centers
from MIDI.midi_to_hit import midi_to_hit

from IPython.display import clear_output

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [2]:
robot_directives = """
directives:

- add_model:
    name: iiwa_right
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [10]
- add_weld:
    parent: world
    child: iiwa_right::iiwa_link_0
    X_PC:
        translation: [0.5, 0.9, 0]
        rotation: !Rpy { deg: [0, 0, -0]}
- add_model:
    name: drum_stick_right
    file: package://robotics_final_project/drum_sticks.sdf
- add_weld:
    parent: iiwa_right::iiwa_link_7
    child: drum_stick_right::stick
    X_PC:
        translation: [0, 0, 0.18]
        rotation: !Rpy { deg: [180, 0, 0]}

- add_model:
    name: iiwa_left
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [10]
- add_weld:
    parent: world
    child: iiwa_left::iiwa_link_0
    X_PC:
        translation: [0.45, 0.3, 0]
        rotation: !Rpy { deg: [0, 0, -0]}
- add_model:
    name: drum_stick_left
    file: package://robotics_final_project/drum_sticks.sdf
- add_weld:
    parent: iiwa_left::iiwa_link_7
    child: drum_stick_left::stick
    X_PC:
        translation: [0, 0, 0.18]
        rotation: !Rpy { deg: [180, 0, 0]}

- add_model:
    name: drum_kit
    file: package://robotics_final_project/drum_kit.sdf

"""

In [3]:
stiffness=20000
dissipation=2

meshcat.Delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=0.00001
    )

parser = Parser(plant)
parser.package_map().Add("robotics_final_project", "./")
parser.AddModelsFromString(robot_directives, ".dmd.yaml")

# plant.mutable_gravity_field().set_gravity_vector([0, 0, 0])

plant.Finalize()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
collision_visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(
        prefix="collision", role=Role.kProximity, visible_by_default=False
    ),
)

kp = [100] * plant.num_positions()
ki = [1] * plant.num_positions()
kd = [20] * plant.num_positions()
iiwa_controller = builder.AddSystem(
    InverseDynamicsController(plant, kp, ki, kd, False)
)
iiwa_controller.set_name("iiwa_controller")

builder.Connect(
    plant.get_state_output_port(),
    iiwa_controller.get_input_port_estimated_state(),
)
builder.Connect(
    iiwa_controller.get_output_port_control(), plant.get_actuation_input_port()
)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)




In [4]:
import numpy as np
from MIDI.midi_to_name import midi_to_name, hit_mapping
from mido import MidiFile, MetaMessage, tick2second
hits = midi_to_hit(filename = 'MIDI/new_midi_2.mid')
print(hits)

[{'hit': 'low_tom', 'last_hit_time': 2.9944459416666667, 'total_time': 2.9944459416666667, 'strength': 87}, {'hit': 'snare', 'last_hit_time': 0.6694447791666667, 'total_time': 3.663890720833333, 'strength': 50}, {'hit': 'low_tom', 'last_hit_time': 0.3388890583333334, 'total_time': 4.002779779166667, 'strength': 71}, {'hit': 'hi_hat', 'last_hit_time': 0.29583348125000003, 'total_time': 4.2986132604166665, 'strength': 51}, {'hit': 'hi_hat', 'last_hit_time': 0.34166683750000004, 'total_time': 4.6402800979166665, 'strength': 127}, {'hit': 'crash', 'last_hit_time': 0.66250033125, 'total_time': 5.302780429166666, 'strength': 127}]


In [5]:
left = ["hi_hat", "crash", "hi_tom", "hi_hat2", "crash2", "hi_tom2","snare","snare2"]
right = ["mid_tom", "ride","mid_tom2", "ride2","low_tom2","low_tom"]
# left = ["hi_hat", "hi_hat2"]
# right = ["snare","snare2"]
def left_right_detector(hit):
    if hit["hit"] in left:
        side = "left"
    else:
        side = "right"
    return side

def angle_constraint_detector(hit,centerx, centery):
    x1 = centers(hit["hit"])[0]
    y1 = centers(hit["hit"])[1]

    vec1 = x1 - centerx
    vec2 = y1 - centery
    
    angle = np.tan(vec1,vec2)

    return angle
 

In [6]:

def visualize_frame(name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies

        Input:
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.

        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )

In [8]:
meshcat.StartRecording()

drum_stick_right = plant.GetModelInstanceByName("drum_stick_right")
drum_stick_left = plant.GetModelInstanceByName("drum_stick_left")
right_frame = plant.GetFrameByName("stick",drum_stick_right )
left_frame = plant.GetFrameByName("stick", drum_stick_left)
avoid_collisions  = True 

for hit in hits:
    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), 10)
    prog = trajopt.get_mutable_prog()
    trajopt.AddDurationCost(1.0)
    trajopt.AddPathLengthCost(1.0)
    trajopt.AddPositionBounds(
        plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()
    )
    trajopt.AddVelocityBounds(
        plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits()
    )
    plant_context = plant.GetMyMutableContextFromRoot(context)

    if left_right_detector(hit) == "right":
        framee = right_frame
    else:
        framee = left_frame

    #get the ending posiiton of the IK solution from the initial position a couple cm above the drumkit target
    end_effector_pose = plant.CalcRelativeTransform(
    plant_context,
    frame_A=plant.world_frame(),
    frame_B=framee
    )

    translation = np.array(centers[hit["hit"]])[:3]
    rotation = RotationMatrix(RollPitchYaw(np.array(centers[hit["hit"]])[3:]))
    X_WD = RigidTransform(rotation, translation)
    visualize_frame('object',X_WD )
    num_q = plant.num_positions()
    q0 = plant.GetPositions(plant_context)

    # start constraint
    start_constraint = PositionConstraint(
        plant,
        plant.world_frame(),
        end_effector_pose.translation(),
        end_effector_pose.translation(),
        framee,
        [0, 0.1, 0],
        plant_context,
    )

    trajopt.AddPathPositionConstraint(start_constraint, 0)
    prog.AddQuadraticErrorCost(
        np.eye(num_q), q0, trajopt.control_points()[:, 0]
    )

    
    goal_constraint = PositionConstraint(
        plant,
        plant.world_frame(),
        X_WD.translation(),
        X_WD.translation(),
        framee,
        [0, 0.1, 0],
        plant_context,
    )

    trajopt.AddPathPositionConstraint(goal_constraint, 1)
    prog.AddQuadraticErrorCost(
        np.eye(num_q), q0, trajopt.control_points()[:, -1]
    )
    # hit_velocity = 0.1*hit['strength']*np.array([1,1,1])

    # plant_auto_diff = plant.ToAutoDiffXd() # make sure to call this after calling plant.finalize
    # final_vel_constraint = SpatialVelocityConstraint(
    #         plant_auto_diff,
    #         plant_auto_diff.world_frame(),
    #         hit_velocity,  # upper limit
    #         hit_velocity,  # lower limit
    #         plant_auto_diff.GetFrameByName("stick",drum_stick_right),
    #         np.array([0, 0, 0]).reshape(-1,1),
    #         plant_auto_diff.CreateDefaultContext(),)

    result = Solve(prog)

    if not result.is_success():
        print("Trajectory optimization failed, even without collisions!")
        print(result.get_solver_id().name())
    trajopt.SetInitialGuess(trajopt.ReconstructTrajectory(result))

    if avoid_collisions:
        # collision constraints
        collision_constraint = MinimumDistanceLowerBoundConstraint(
            plant, 0.01, plant_context, None, 0.1
        )
        evaluate_at_s = np.linspace(0, 1, 25)
        for s in evaluate_at_s:
            trajopt.AddPathPositionConstraint(collision_constraint, s)
        result = Solve(prog)
        if not result.is_success():
            print("Trajectory optimization failed")
            print(result.get_solver_id().name())
        else:
            print("sucess")
    print(trajopt.ReconstructTrajectory(result), context, plant, visualizer)
    PublishPositionTrajectory(
        trajopt.ReconstructTrajectory(result), context, plant, visualizer
    )
    collision_visualizer.ForcedPublish(
        collision_visualizer.GetMyContextFromRoot(context)
    )
    # simulator.AdvanceTo(hit["total_time"])
    

# simulator.AdvanceTo(10)
meshcat.StopRecording() 
meshcat.PublishRecording()

sucess
<pydrake.trajectories.BsplineTrajectory object at 0x116d49ab0> ::_ Context (of a Diagram)
---------------------------
14 total continuous states
28 total discrete states in 1 groups
708 total numeric parameters in 89 groups
28 total abstract parameters

::_::plant Context
-------------------
Time: 0.0820046
States:
  1 discrete state groups with
     28 states
        -1.907624620811826 -0.2020590195641363  -0.977456605939121  -1.447013173411607 -0.6158524618971967   1.613994762534955   3.044117320406445 -0.2430973316275392 -0.9527330595809287  -1.704075434999719 -0.4723618295964256 -0.8627350950723481  0.2031602331485675    3.05432941847408                   0                   0                   0                   0                   0                   0                   0                   0                   0                   0                   0                   0                   0                   0

Parameters:
  88 numeric parameter groups with
     1 paramete